In [1]:
import sys
import os
import time
import sys
import os
import time
import numpy as np
from os import environ, listdir, makedirs
from os.path import dirname, exists, expanduser, isdir, join, splitext
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import nltk 
import string
import pandas as pd
from nltk.corpus import stopwords
from senticnet.senticnet import Senticnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk import sent_tokenize, word_tokenize, pos_tag
from random import uniform

sn = Senticnet()

In [2]:
if __name__ == '__main__':
    with open('C:\\PATH\\SAMPLE.txt','r') as old, open('C:\\PATH\\DATASET.txt','w') as new:
        for line in old:
            line = line.replace('"', "'")
            new.write(line)
    with open('C:\\PATH\\DATASET.txt') as nuovo:
        result = list(list(l for l in e.split("','") if l) for e in nuovo.read().split("\n")) 
    result = [x for x in result if not x == []]
    result = [x for x in result if not "None'" in x]
    #remove duplicate
    df = pd.DataFrame(result)
    df = df.drop_duplicates()
    result = df.values.tolist()
    for i in range(len(result)):
        if "{'basic': 'Bullish'}'" in result[i][3]:
             result[i][3] = "positive"
        else:
            result[i][3] = "negative"
    #using bonzanini as base  
    import re
    emoticons_str = r"""
        (?:
            [:=;] # Eyes
            [oO\-]? # Nose (optional)
            [D\)\]\(\]/\\OpP] # Mouth
        )"""
    regex_str = [
        emoticons_str,
        r'(?:[\.]{2,10})', 
        r'(?:[Ss]+[&]+[Pp]+)',
        r'\\(?:[a-z0-9]{3,3}[\\])+[a-z0-9]{3,3}', 
        r"(?:\$+[\w_]*[.]?[\d]*)", 
        r'\\n\\n',
        r'\\n',
        r'<[^>]+>', # HTML tags
        r'(?:@[\w_]+)', # @-mentions
        r"(?:\#+[\w_]+[\w\'_\-.]*[\w_]+)", # hash-tags
        r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
        r'(?:(?:\.?\d+,?-?%?\$?)+(?:\.?\/?\d+\/?\d*)?(?:[cb])?)', # numbers
        r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
        r'(?:[\w_]+)', # other words
        r'(?:\S)' # anything else
    ]

    tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
    emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
    def tokenize(s):
        return tokens_re.findall(s)
 
    def preprocess(s, lowercase=True):
        tokens = tokenize(s)
        if lowercase:
            tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
        return tokens
    punctuation = ['"', '#', '$', '%', '(', ')', '*', '-', ',', '/', '<', '>', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
    stop = punctuation + ['\\n', 'etc']
    
    new_sentiment = {}
    lista = list(sn.data.keys())
    lemmas = set(lista)    
    train_data = []
    train_labels = []
    test_data = []
    test_labels = []
    list_corpus = []
    
    lemmatizer = WordNetLemmatizer()
    def penn_to_wn(tag):
        
        if tag.startswith('J'):
            return wn.ADJ
        elif tag.startswith('N'):
            return wn.NOUN
        elif tag.startswith('R'):
            return wn.ADV
        elif tag.startswith('V'):
            return wn.VERB
        return None

    def add_words(lemma):
        count_posi = 0
        count_nega = 0        
        for i in range(len(result)):
            ID = result[i][0]
            tweet = result[i][2]
            label = result[i][3]
            microtext(tweet)
            terms_stop = [term for term in preprocess(tweet) if term not in stop and not term.startswith(('http', '\\x', '#'))]
            tagged_sentence = pos_tag(terms_stop)
            for word, tag in tagged_sentence:
                wn_tag = penn_to_wn(tag)
                if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                    continue
 
                lemmat = lemmatizer.lemmatize(word, pos=wn_tag)
                if not lemmat:
                    continue
                if lemma == lemmat:
                    if label == 'positive':
                        count_posi += 1
                    else:
                        count_nega += 1
                else:
                    continue
        unbalance_ratio = 5
        if count_posi > (count_nega*unbalance_ratio) and count_posi > unbalance_ratio:
            new_sentiment[lemma] = 0.5
            lemmas.add(lemma)
        elif (count_posi*unbalance_ratio) < count_nega and count_nega > unbalance_ratio:
            new_sentiment[lemma] = -0.5
            lemmas.add(lemma)        
        return None
    
    #compute the percentage of right prediction of tweets that have the specific word. The word here has the score not updated
    def baseline_performance(baseline_word):
        right_1 = 0
        wrong_1 = 0
        for lll in range(len(test_data)):
            check_word_in_sentence = []
            tagged_sentence = pos_tag(test_data[lll])
            for word, tag in tagged_sentence:
                wn_tag = penn_to_wn(tag)
                if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                    continue
                lemma = lemmatizer.lemmatize(word, pos=wn_tag)
                if not lemma:
                    continue
                check_word_in_sentence.append(lemma)
            if baseline_word in check_word_in_sentence:
                compare = test_labels[lll]
                baseline_word_score = np.zeros(VOCABULARY_SIZE)
                for yyy in check_word_in_sentence:
                    if yyy in list_new:
                        sentiment = 0.0
                        sentiment = sn.polarity_intense(yyy)
                        if yyy in new_sentiment.keys():
                            sentiment += new_sentiment[yyy]
                        if yyy in CHANGE_IN_THE_LEXICON.keys():
                            sentiment = CHANGE_IN_THE_LEXICON[yyy]
                        indexes = word_index[yyy]
                        baseline_word_score[indexes] = sentiment
                    else:
                        continue
                prova = []
                prova.append(baseline_word_score)
                prediction_liblinear = classifier_liblinear.predict(prova)
                if compare == prediction_liblinear:
                    right_1 += 1
                else:
                    wrong_1 += 1    
            else:
                continue
        baseline_performance.performance_baseline = (right_1/(right_1+wrong_1))
        print('baseline perf: %f' %(baseline_performance.performance_baseline))        
        return None
                    
    def choose_the_best_arm(position):
        already_modified = {}
        for j in range(len(position)):
            baseline_word = list_new[position[j]]
            already_modified[baseline_word] = False
            if baseline_word in CHANGE_IN_THE_LEXICON.keys():
                #narrow the interval and the number of max interations cosi tengo conto (parzialmente anche di tutte gli altri cambiamenti che sto facendo => considere le interdipendenze tra le parole)
                already_modified[baseline_word] = True
                continue
            else:
                for h in range(0,11):
                    old_score = test_word_score[position[j]]
                    new_score = old_score + uniform(-1,1)
                    performance_baseline = 0.0
                    baseline_performance(baseline_word)
                    right = 0 
                    wrong = 0
                    performance = 0.0
                        #change the polarity score of the word
                    for ll in range(len(test_data)):
                        check_word_in_sentence = []
                        tagged_sentence = pos_tag(test_data[ll])
                        for word, tag in tagged_sentence:
                            wn_tag = penn_to_wn(tag)
                            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                                continue
                            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
                            if not lemma:
                                continue
                            check_word_in_sentence.append(lemma)
                        if list_new[position[j]] in check_word_in_sentence:
                                #print('ce un altro errore')
                            compare = test_labels[ll]
                            new_test_word_score = np.zeros(VOCABULARY_SIZE)
                            for yy in check_word_in_sentence:
                                if yy in list_new:
                                    sentiment = 0.0
                                    if yy == list_new[position[j]]:
                                        sentiment = new_score
                                    else:
                                        sentiment = sn.polarity_intense(yy)
                                        if yy in new_sentiment.keys():
                                            sentiment += new_sentiment[yy]
                                        if yy in CHANGE_IN_THE_LEXICON.keys():
                                            sentiment = CHANGE_IN_THE_LEXICON[yy]
                                    indexes = word_index[yy]
                                    new_test_word_score[indexes] = sentiment
                                else:
                                    continue
                            prova = []
                            prova.append(new_test_word_score)
                            prediction_liblinear = classifier_liblinear.predict(prova)
                            if compare == prediction_liblinear:
                                right += 1
                            else:
                                wrong += 1    
                        else:
                            continue
                    performance = (right/(right+wrong))
                    if performance > baseline_performance.performance_baseline:
                        word_with_new_score = list_new[position[j]]
                        ALREADY_MODIFIED[word_with_new_score] = 1
                        change_in_the_lexicon[word_with_new_score] = new_score #cosi cosnidero i cambiamenti solo da un tweet all'altro e non per ogni parola
                        break
                    else:
                        continue
        for g in already_modified:
            if already_modified[g] == False:
                continue
            else:
                ALREADY_MODIFIED[g] += 1
                if ALREADY_MODIFIED[g] > 10:
                    continue
                else:
                    for w in range(0,11-ALREADY_MODIFIED[g]):
                        old_score = CHANGE_IN_THE_LEXICON[g]
                        new_score = old_score + uniform(-1/ALREADY_MODIFIED[g],1/ALREADY_MODIFIED[g])
                        performance_baseline = 0.0
                        baseline_word = g
                        baseline_performance(baseline_word)
                        right = 0 
                        wrong = 0
                        performance = 0.0
                            #change the score of the word
                        for ll in range(len(test_data)):
                            check_word_in_sentence = []
                            tagged_sentence = pos_tag(test_data[ll])
                            for word, tag in tagged_sentence:
                                wn_tag = penn_to_wn(tag)
                                if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                                    continue
                                lemma = lemmatizer.lemmatize(word, pos=wn_tag)
                                if not lemma:
                                    continue
                                check_word_in_sentence.append(lemma)
                            if g in check_word_in_sentence:
                                #print('ce un altro errore')
                                compare = test_labels[ll]
                                new_test_word_score = np.zeros(VOCABULARY_SIZE)
                                for yy in check_word_in_sentence:
                                    if yy in list_new:
                                        sentiment = 0.0
                                        if yy == g:
                                            sentiment = new_score
                                        else:
                                            sentiment = sn.polarity_intense(yy)
                                            if yy in new_sentiment.keys():
                                                sentiment += new_sentiment[yy]
                                            if yy in CHANGE_IN_THE_LEXICON.keys():
                                                sentiment = CHANGE_IN_THE_LEXICON[yy]
                                        indexes = word_index[yy]
                                        new_test_word_score[indexes] = sentiment
                                    else:
                                        continue
                                prova = []
                                prova.append(new_test_word_score)
                                prediction_liblinear = classifier_liblinear.predict(prova)
                                if compare == prediction_liblinear:
                                    right += 1
                                else:
                                    wrong += 1    
                            else:
                                continue
                        performance = (right/(right+wrong))
                        if performance > baseline_performance.performance_baseline:
                            word_with_new_score = g
                            change_in_the_lexicon[word_with_new_score] = new_score #cosi cosnidero i cambiamenti solo da un tweet all'altro e non per ogni parola
                            break
                        else:
                            continue
                            
        return None
    
    for i in range(len(result)):
        ID = result[i][0]
        microtext(tweet)
        terms_stop = [term for term in preprocess(tweet) if term not in stop and not term.startswith(('http', '\\x', '#'))]
        tagged_sentence = pos_tag(terms_stop)
        sentence_lemmas = []
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                continue 
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            list_corpus.append(lemma)
            sentence_lemmas.append(lemma)
            
        corpus = set(list_corpus)
        set_lemmas = set(sentence_lemmas)
        #add words to sent lex
        if set_lemmas & lemmas:
            continue
        else: 
            for word, tag in tagged_sentence:
                wn_tag = penn_to_wn(tag)
                if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                    continue
 
                lemma = lemmatizer.lemmatize(word, pos=wn_tag)
                if not lemma:
                    continue
                if wn_tag in (wn.VERB, wn.ADJ) and lemma not in lemmas:
                    add_words(lemma) 
                    print(lemma)
                else:
                    continue

    new = lemmas.intersection(corpus)
    list_new = list(new) 
    word_index = {w: idx for idx, w in enumerate(list_new)}  
    VOCABULARY_SIZE = len(list_new)  
    print('VOCABULARY_SIZE %f' % (VOCABULARY_SIZE))
    
    count_pos = 0
    count_neg = 0    
    real_train = []
    real_test = []
    #split test and training data
    for n in range(len(result)):
        ID = result[n][0]
        tweet = result[n][2]
        label = result[n][3]
        microtext(tweet)
        terms_stop = [term for term in preprocess(tweet) if term not in stop and not term.startswith(('http', '\\x', '#'))]
        if label == 'positive':
            count_pos += 1
            if count_pos < 1001:
                test_data.append(terms_stop)
                test_labels.append(label)
            else:
                train_data.append(terms_stop)
                train_labels.append(label)    
        else:
            count_neg += 1
            if count_neg < 1001:
                test_data.append(terms_stop)
                test_labels.append(label)
            else:
                train_data.append(terms_stop)
                train_labels.append(label)
                 
    for j in range(len(train_data)):
        train_word_score = np.zeros(VOCABULARY_SIZE)
        tagged_sentence = pos_tag(train_data[j])
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                continue
 
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue

            if lemma in list_new: 
            # Take the first sense, the most common
                sentiment = 0.0   
                sentiment = sn.polarity_intense(lemma)
                if lemma in new_sentiment.keys():
                    sentiment += new_sentiment[lemma]
                else:
                    pass
                indexes = word_index[lemma]
                train_word_score[indexes] = sentiment
            else:
                continue
            
        real_train.append(train_word_score)
        
    contiamo = 0
    CHANGE_IN_THE_LEXICON = {}
    ALREADY_MODIFIED = {}
    for l in range(len(test_data)): 
        compare = test_labels[l]
        test_word_score = np.zeros(VOCABULARY_SIZE)
        temp_test_word_score = np.zeros(VOCABULARY_SIZE)
        tagged_sentence = pos_tag(test_data[l])
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                continue
 
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
 
            if lemma in list_new: 
            # Take the first sense, the most common
                sentiment = 0.0   
                sentiment = sn.polarity_intense(lemma)
                if lemma in new_sentiment.keys():
                    sentiment += new_sentiment[lemma] 
                if lemma in CHANGE_IN_THE_LEXICON.keys():
                    sentiment = CHANGE_IN_THE_LEXICON[lemma] 
                indexes = word_index[lemma]
                test_word_score[indexes] = sentiment
                temp_test_word_score[indexes] = sentiment
            else:
                continue
        prova = []
        prova.append(test_word_score)
        list_rewards = []
        classifier_liblinear = svm.LinearSVC()
        classifier_liblinear.fit(real_train, train_labels)
        prediction_liblinear = classifier_liblinear.predict(prova)
        if compare == prediction_liblinear:
            continue
        else:
            contiamo +=1
            position = []
            for q in range(len(test_word_score)):
                if test_word_score[q] != 0:
                    position.append(q)
                else:
                    continue
            change_in_the_lexicon = {}
            choose_the_best_arm(position)
            CHANGE_IN_THE_LEXICON.update(change_in_the_lexicon)
            change_in_the_lexicon = {}
            
    print('quante sbaglaite cerano', contiamo) 
    print(CHANGE_IN_THE_LEXICON)
    #update the sentiment lexicon
    final_real_train = []
    final_real_test = []        
    
    for j in range(len(train_data)):
        final_train_word_score = np.zeros(VOCABULARY_SIZE)
        tagged_sentence = pos_tag(train_data[j])
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                continue
 
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue

            if lemma in list_new: 
            # Take the first sense, the most common
                sentiment = 0.0   
                sentiment = sn.polarity_intense(lemma)
                if lemma in new_sentiment.keys():
                    sentiment += new_sentiment[lemma]
                if lemma in CHANGE_IN_THE_LEXICON.keys():
                    sentiment = CHANGE_IN_THE_LEXICON[lemma]
                indexes = word_index[lemma]
                final_train_word_score[indexes] = sentiment
            else:
                continue
            #if all(p == 0.0 for p in temp):
                #print(1)
            #else: 
                #continue
            
        final_real_train.append(final_train_word_score)

    for l in range(len(test_data)):      
        final_test_word_score = np.zeros(VOCABULARY_SIZE)
        tagged_sentence = pos_tag(test_data[l])
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                continue
 
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
 
            if lemma in list_new: 
            # Take the first sense, the most common
                sentiment = 0.0   
                sentiment = sn.polarity_intense(lemma)
                if lemma in new_sentiment.keys():
                    sentiment += new_sentiment[lemma]
                if lemma in CHANGE_IN_THE_LEXICON.keys():
                    sentiment = CHANGE_IN_THE_LEXICON[lemma]
                indexes = word_index[lemma]
                final_test_word_score[indexes] = sentiment             
            else:
                continue
        final_real_test.append(final_test_word_score)
        
    # Perform classification with SVM, kernel=rbf
    new_classifier_rbf = svm.SVC()
    t0 = time.time()
    new_classifier_rbf.fit(final_real_train, train_labels)
    t1 = time.time()
    new_prediction_rbf = new_classifier_rbf.predict(final_real_test)
    t2 = time.time()
    time_rbf_train = t1-t0
    time_rbf_predict = t2-t1

    # Perform classification with SVM, kernel=linear
    new_classifier_linear = svm.SVC(kernel='linear')
    t0 = time.time()
    new_classifier_linear.fit(final_real_train, train_labels)
    t1 = time.time()
    new_prediction_linear = new_classifier_linear.predict(final_real_test)
    t2 = time.time()
    time_linear_train = t1-t0
    time_linear_predict = t2-t1

    # Perform classification with SVM, kernel=linear
    new_classifier_liblinear = svm.LinearSVC()
    t0 = time.time()
    new_classifier_liblinear.fit(final_real_train, train_labels)
    t1 = time.time()
    new_prediction_liblinear = new_classifier_liblinear.predict(final_real_test)
    t2 = time.time()
    time_liblinear_train = t1-t0
    time_liblinear_predict = t2-t1

    # Print results in a nice table
    print("Results for SVC(kernel=rbf)")
    print("Training time: %fs; Prediction time: %fs" % (time_rbf_train, time_rbf_predict))
    print(classification_report(test_labels, new_prediction_rbf))
    print("Accuracy: %f" % (accuracy_score(test_labels, new_prediction_rbf)))
    print("\n")
    print("Results for SVC(kernel=linear)")
    print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
    print(classification_report(test_labels, new_prediction_linear))
    print("Accuracy: %f" % (accuracy_score(test_labels, new_prediction_linear)))
    print("\n")
    print("Results for LinearSVC()")
    print("Training time: %fs; Prediction time: %fs" % (time_liblinear_train, time_liblinear_predict))
    print(classification_report(test_labels, new_prediction_liblinear))
    print("Accuracy: %f" % (accuracy_score(test_labels, new_prediction_liblinear)))

VOCABULARY_SIZE 3981.000000
quante sbaglaite cerano 500
{'kid': -0.74119883051033664, 'signal': -0.71086318306468343, 'week': 0.48406910558510818, 'power': 0.30960153109099808, 'leave': 0.78638790724139651, 'phone': 1.0814030461171646, 'use': -0.46118716248958841, 'pixel': 0.77794323243479502, 'down': -0.74584700847119767, 'trade': -0.20462027410042133, 'amazon': -0.31709513091556751, 'sit': -0.88200951145746953, 'wed': -0.08725524967624787, 'break': 1.1326904044950026, 'close': -0.24697661405293775, 'sell': 0.92887726375241086, 'time': 0.33496908100668749, 'least': -1.0879622537879472, 'change': -0.4468375794248981, 'pro': 1.0748711912969571, 'battery': 0.45108123892942165, 'cheap': -0.67781095464126995, 'ipod': -0.26988408029350758, 'take': -0.41820263423651549, 'hole': 1.7227888616800606, 'death': 0.28067158883927112, 'revenue': -0.74915048064869549, 'less': -1.6901861374806353, 'make': -0.33712784509531912, 'weekend': -0.38818454354514198, 'buddy': 1.0296066073358305, 'dump': -0.50

C:\Users\zxing001\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
